# Deploy

Utilities to deploy app

In [ ]:
# default_exp deploy.app

In [ ]:
# export
import json
import requests
from dataclasses import dataclass, field
from importlib.machinery import SourceFileLoader
from importlib.util import spec_from_loader, module_from_spec
from pathlib import Path
from textwrap import dedent
from typing import Any, Union

from black import Mode, format_file_contents
from jinja2 import Template, DebugUndefined


In [ ]:
# exporti
PathStr = Union[Path, str]

## Deploy with Streamlit and NGROK

In [ ]:
# export
def deploy_app(app="app.py", from_jupyter=True):
    """Deploy a streamlit app using ngrok"""
    if not Path(app).is_file():
        print(f"ERROR: the app {app} does not exist!")
        return

    if from_jupyter:
        get_ipython().system_raw('ngrok http 8501 &')
        resp = requests.get("http://localhost:4040/api/tunnels")
        tunnel = json.loads(resp.content)["tunnels"][0]
        local = tunnel["config"]["addr"]
        public = tunnel["public_url"]
    else:
        raise NotImplementedError(f"Deployment outside jupyter currently not supported")

    print(dedent(f"""\
        Let's create a tunnel to port {local.split(":")[-1]}!
        1. Create in a cell : !nohup streamlit run {app}
        2. Run that cell
        3. Click on this link: {public}

        Note: we recommend to put the the code `!nohup streamlit ...`
        after this line of code (instead of a new dedicated cell).
    """))

## Utility functions to load a module

In [ ]:
# export
def load_module(module_path: PathStr, module_name="module") -> Any:
    """Load and return a module"""
    module_path = Path(module_path)
    if not (module_path.is_file() and module_path.suffix == ".py"):
        raise AttributeError(f"Module {module_path} not found or not a .py file")
    loader = SourceFileLoader(module_name, str(module_path))
    spec = spec_from_loader(loader.name, loader)
    if not spec:
        raise AttributeError(f"Empty module spec from {module_path}")
    module = module_from_spec(spec)
    loader.exec_module(module)
    return module


def import_from_module(module_path: PathStr, name: str, module_name="module") -> Any:
    """Import a list of names from a module from its path"""
    module = load_module(module_path, module_name=module_name)
    try:
        return getattr(module, name)
    except AttributeError:
        raise AttributeError(f"Name {name} not found in {module_path}") from None


## Get template of App

In [ ]:
# export
def blackify(code:str) -> str:
    """Run "black" on a code passed as string"""
    return format_file_contents(code, mode=Mode(), fast=False)

### Base Template

In [ ]:
# exporti
TEMPLATE_FLOW = """
from pathlib import Path

import streamlit as st

{{ specific_import }}
{{ custom_import }}

st.set_page_config(page_title="ML deployment, by unpackAI", page_icon="🚀")
st.image("https://unpackai.github.io/unpackai_logo.svg")
st.title("{{ title }}")
st.write("*by {{ author }}*")
st.write("---")

{{ load_model }}

{{ display_prediction }}

{{ input_2_prediction }}
"""


@dataclass
class TemplateCode:
    specific_import: str = ""
    load_model: str = ""
    display_prediction: str = ""
    input_2_prediction: str = """\
        select = st.sidebar.radio("How to load {{ input }}?", ["from file{% if multiple %}s{% endif %}", "from URL"])
        st.sidebar.write("---")

        if select == "from URL":
            url = st.sidebar.text_input("url")
            if url:
                display_prediction(url)

        else:
            {% if multiple -%}
            files = st.sidebar.file_uploader("Choose {{ input }}", accept_multiple_files=True)
            for file in files:  # type:ignore # this is an iterable
                display_prediction(file)
            {% else -%}
            file = st.sidebar.file_uploader("Choose {{ input }}")
            if file:
                display_prediction(file)
            {% endif %}
    """


### Templates for CSV

In [ ]:
# exporti

# NOTE: We need re-implementation of elements defined during loading

TEMPLATE_CV_FASTAI = TemplateCode(
    specific_import="""
        {# The ✨ below is to go around import modification by nbdev #}
        from {# ✨ #}unpackai.deploy.cv import get_image, get_learner, dummy_function
    """
        ,
    load_model="""\
        {{ implem_4_model }}

        learn = get_learner(Path(__file__).parent / "{{ model }}")
        vocab = learn.dls.vocab
    """,
    display_prediction="""
        def display_prediction(pic):
            img = get_image(pic)
            with learn.no_bar():
                prediction, idx, probabilities = learn.predict(img)
            col_img, col_pred = st.columns(2)
            col_img.image(img, caption=getattr(pic, "name", None))
            col_pred.write(f"### {prediction}")
            col_pred.metric(f"Probability", f"{probabilities[idx].item()*100:.2f}%")
    """,
)


### Templates for Tabular

In [ ]:
# exporti

# NOTE: Loading model and prediction depends on the module (e.g. "regression")

TEMPLATE_TABULAR_PYCARET = TemplateCode(
    specific_import="""
        import pandas as pd
        from pycaret.{{ module }} import load_model, predict_model
    """,
    load_model="""model = load_model("{{ model }}")""",
    display_prediction="""
        def display_prediction(csv):
            df = pd.read_csv(csv)
            predictions = predict_model(model, data = df)
            st.dataframe(predictions)
    """,
)


### Templates for NLP 

In [ ]:
# exporti
TEMPLATE_NLP_HF = TemplateCode(
    specific_import="""
        # TODO
    """,
    load_model="""
        # TODO
    """,
    display_prediction="""
        def display_prediction(csv:PathStr):
            pass # TODO
    """,
)


### Class to Create Templates

In [ ]:
# exporti

LIST_TEMPLATES = {
    "CV": {"fastai": TEMPLATE_CV_FASTAI},
    "Tabular": {"pycaret": TEMPLATE_TABULAR_PYCARET},
    "NLP": {"hugging_face": TEMPLATE_NLP_HF},
}

In [ ]:
# export
@dataclass
class StreamlitApp:
    """Class to generate Streamlit App for different applications

    Available applications: "CV", "Tabular", "NLP", and "Custom"
    """

    application: str
    framework: str = field(init=False, default="")
    content: str = field(init=False, default="")
    _title: str = field(init=False, default="Streamlit App")

    def __post_init__(self):
        applications = list(LIST_TEMPLATES)
        if self.application not in applications:
            raise AttributeError(
                f"ERROR: Application {self.application} not supported. "
                f"Available applications: {applications}"
            )

    @property
    def template(self) -> Template:
        """Get the template"""
        base_template = Template(TEMPLATE_FLOW, undefined=DebugUndefined)
        try:
            template_framework = LIST_TEMPLATES[self.application][self.framework]
        except KeyError as missing:
            raise AttributeError(f"Element {missing} not found among list of templates")

        return Template(
            base_template.render(
                specific_import=dedent(template_framework.specific_import),
                load_model=dedent(template_framework.load_model),
                display_prediction=dedent(template_framework.display_prediction),
                input_2_prediction=dedent(template_framework.input_2_prediction),
            )
        )

    def _render(
        self,
        framework: str,
        title: str,
        author: str,
        model: PathStr,
        custom_import: str = "",
        **kwargs,
    ):
        """Generic function for rendering"""
        self.framework = framework
        self._title = title
        input_ = {
            "CV": "images",
            "Tabular": "CSV",
            "NLP": "Text",
        }.get(self.application, "input")
        self.content = self.template.render(
            title=title,
            author=author,
            model=model,
            input=input_,
            multiple=(input_.endswith("s")),
            custom_import=custom_import,
            **kwargs,
        )
        self.content = blackify(self.content)
        return self

    def render_fastai(
        self,
        title: str,
        author: str,
        model: PathStr,
        implem_4_model: str,
        custom_import: str = "",
    ) -> "StreamlitApp":
        """Render an app based on template

        Args:
            title: title of the App
            author: author of the App
            model: path of .pkl model to load (exported with `learn.export(...)`)
            implem_4_model: extra implementations needed to load the model
                (e.g. function used for labelling)
            custom_import: custom import (optional)
        """
        return self._render(
            "fastai",
            title,
            author,
            model,
            implem_4_model=dedent(implem_4_model),
            custom_import=custom_import,
        )

    def render_pycaret(
        self,
        title: str,
        author: str,
        model: PathStr,
        module: str,
        custom_import: str = "",
    ) -> "StreamlitApp":
        """Render an app based on template

        Args:
            title: title of the App
            author: author of the App
            model: path of .pkl model to load (exported with `learn.export(...)`)
            module: regression, classification, etc.
            custom_import: custom import (optional)
        """
        return self._render(
            "pycaret",
            title,
            author,
            model.replace(".pkl", ""),
            module=module,
            custom_import=custom_import,
        )

    def append(self, content: str) -> "StreamlitApp":
        """Add additional content to the app"""
        self.content += dedent(content)
        return self

    def save(self, dest: PathStr, show=False):
        """Write the app to a file"""
        Path(dest).write_text(self.content, encoding="utf-8")
        print(f"Saved app '{self._title}' to '{dest}'")
        if show:
            print("-" * 20)
            print(self.content)


### Custom App

In [ ]:
# exporti
TEMPLATE_CUSTOM = TemplateCode(
    specific_import="{{ specific_import }}",
    load_model="",
    display_prediction="",
    input_2_prediction="{{ make_predictions }}"
)

In [ ]:
# export
@dataclass
class StreamlitAppCustom:
    """Application with custom implementation"""

    content: str = field(init=False, default="")
    _title: str = field(init=False, default="Streamlit App")

    def render(
        self,
        title: str,
        author: str,
        make_predictions: Union[str, Path],
        custom_import: str = "",
    ) -> "StreamlitAppCustom":
        """Render an app based on template

        Args:
            title: title of the App
            author: author of the App
            make_predictions: custom implementations, either code or file path
                if a file is provided, it shall implement the function
                'def make_predictions()'
            custom_import: custom import (optional)
        """
        self._title = title
        base_template = Template(TEMPLATE_FLOW, undefined=DebugUndefined)
        template = Template(
            base_template.render(
                specific_import=dedent(TEMPLATE_CUSTOM.specific_import),
                load_model=dedent(TEMPLATE_CUSTOM.load_model),
                display_prediction=dedent(TEMPLATE_CUSTOM.display_prediction),
                input_2_prediction=dedent(TEMPLATE_CUSTOM.input_2_prediction),
            )
        )

        if isinstance(make_predictions, str) and not make_predictions.endswith(".py"):
            self.content = template.render(
                title=title,
                author=author,
                specific_import="",
                make_predictions=dedent(make_predictions),
                custom_import=custom_import,
            )
        elif isinstance(make_predictions, Path) or make_predictions.endswith(".py"):
            loaded_make_predictions = f"""\
                make_predictions = import_from_module(r"{make_predictions}", "make_predictions")
                make_predictions()
            """
            self.content = template.render(
                title=title,
                author=author,
                specific_import="from unpackai.deploy.app import import_from_module",
                make_predictions=dedent(loaded_make_predictions),
                custom_import=custom_import,
            )
        else:
            raise AttributeError(
                f"Incorrect type for {make_predictions}: shall be code or .py path"
            )

        self.content = blackify(self.content)
        return self

    def save(self, dest: PathStr, show=False):
        """Write the app to a file"""
        Path(dest).write_text(self.content, encoding="utf-8")
        print(f"Saved app '{self._title}' to '{dest}'")
        if show:
            print("-" * 20)
            print(self.content)

## Test

In [ ]:
# hide
# To be able to run the tests in the Notebook
from pathlib import Path
import ipytest
import sys

ipytest.autoconfig()

root_dir = Path("..").resolve()
sys.path.append(str(root_dir / "test"))

In [ ]:
# exportest
# For Test Cases (might have duplicate import because it will be in a dedicated file)
import re
from pathlib import Path
from typing import List

import pytest
from test_common.utils_4_tests import DATA_DIR, diff_files

DEPLOY_DATA_DIR = DATA_DIR / "deploy"


In [ ]:
# exportest
def test_streamlit_cv_fastai(tmp_path):
    """Test Streamlit app for CV with fastai"""
    dest = tmp_path / "app_cv_fastai.py"
    StreamlitApp("CV").render_fastai(
        title="Is it a cat?",
        author="Jeff",
        model="model.pkl",
        implem_4_model="is_cat = dummy_function",
    ).append(
        """
        st.sidebar.write("---")
        st.sidebar.button("Show Balloons", on_click=st.balloons)
        """
    ).save(
        dest=dest
    )

    assert diff_files(DEPLOY_DATA_DIR / "app_CV.py", dest, show=True) == ""


In [ ]:
# exportest 
def test_streamlit_tabular_pycaret(tmp_path):
    """Test Streamlit app for Tabular with pycaret"""
    dest = tmp_path / "app_tabular_pycaret.py"
    StreamlitApp("Tabular").render_pycaret(
        title="My Mini Tabular App",
        author="Jeff",
        model="model.pkl",
        module="regression",
    ).save(dest=dest)

    assert diff_files(DEPLOY_DATA_DIR / "app_Tabular.py", dest, show=True) == ""


In [ ]:
# exportest
def test_custom_app_code(tmp_path):
    """Test custom app with code provided via string"""
    dest = tmp_path / "app_custom_code.py"
    code = """
        st.write("hello")
        st.balloons()
    """
    StreamlitAppCustom().render(
        "Custom App", "Jeff", code
    ).save(dest=dest)

    assert diff_files(DEPLOY_DATA_DIR / "app_custom.py", dest, show=True) == ""

In [ ]:
# exportest
def test_custom_app_external(tmp_path):
    """Test custom app with code provided via string"""
    dest = tmp_path / "app_custom_external.py"

    StreamlitAppCustom().render(
        "Custom App", "Jeff", DEPLOY_DATA_DIR / "correct_code_for_custom_app.py"
    ).save(dest=dest)

    # Because we have a absolute path of "test_data/deploy", we will replace by relative path
    # otherwise our tests will always be failed
    content = dest.read_text(encoding="utf-8").replace(str(DEPLOY_DATA_DIR), "<deploy_dir>")
    content = content.replace(">\\", ">/").replace('r"<deploy_dir>/', 'Path(__file__).parent / "')
    dest.write_text(content, encoding="utf-8")

    assert diff_files(DEPLOY_DATA_DIR / "app_custom_external.py", dest, show=True) == ""


In [ ]:
ipytest.run()

....                                                                                         [100%]##vso[results.publish type=JUnit;runTitle='Pytest results';]e:\AnsysDev\_perso_repo\unpackai\nbs\test-output.xml
##vso[task.logissue type=warning;]Coverage XML was not created, skipping upload.

------------ generated xml file: e:\AnsysDev\_perso_repo\unpackai\nbs\test-output.xml -------------
4 passed in 0.27s
